# Watson OpenScale Mortgage Default Lab - Model Feed

Part two of a hands-on lab for IBM Watson OpenScale, this notebook should be run in a Watson Studio project with Python 3.6 or greater. This notebook will use the Watson Machine Learning client to send scoring requests to the model. You can specify the number of requests sent in the cells below.

Replace the credentials below with those of your Watson Machine Learning instance with the mortgage default model deployed.

In [ ]:
WML_CREDENTIALS = {
  "apikey": "xxxxxxxxx",
  "iam_apikey_description": "Auto-generated for key xxxxxxxx",
  "iam_apikey_name": "openscale-credentials",
  "iam_role_crn": "crn:v1:bluemix:public:iam::::serviceRole:Writer",
  "iam_serviceid_crn": "crn:v1:bluemix:public:iam-identity::xxxxxxxxx",
  "instance_id": "xxxxxxx",
  "url": "https://us-south.ml.cloud.ibm.com"
}

In the cell below, ensure that the model and deployment names are the same used for creating the model, and set the minimum and maximum number of records you wish to send to the model for scoring.

In [ ]:
MODEL_NAME = 'Mortgage Default'
DEPLOYMENT_NAME = 'Mortgage Default - Production'

MIN_PAYLOADS = 50
MAX_PAYLOADS = 75

Using the credentials from above, create a Watson Machine Learning API client.

In [ ]:
from watson_machine_learning_client import WatsonMachineLearningAPIClient

client = WatsonMachineLearningAPIClient(WML_CREDENTIALS)

Get the deployment and model IDs from the WML client.

In [ ]:
mortgage_deployment_id = None
model_deployment_ids = client.deployments.get_uids()
for deployment_id in model_deployment_ids:
    deployment = client.deployments.get_details(deployment_id)
    model_id = deployment['entity']['deployable_asset']['guid']
    if deployment['entity']['name'] == DEPLOYMENT_NAME:
        print('Deployment id:', deployment_id)
        mortgage_deployment_id = deployment_id
        print('Model id:', model_id)
        break

Get the scoring URL of the model.

In [ ]:
deployment_details = client.deployments.get_details(mortgage_deployment_id)
scoring_url = deployment_details['entity']['scoring_url']
scoring_url

Download the JSON records from github.

In [ ]:
!rm mortgage_feed.json
!wget https://raw.githubusercontent.com/emartensibm/mortgage-default/master/mortgage_feed.json

In [ ]:
import json

with open('mortgage_feed.json', 'r') as scoring_file:
    data = json.load(scoring_file)

Get the feature fields from the data file, dropping the transaction ID.

In [ ]:
data['fields'][1:]

Set up the data structure for scoring requests. The ID field was not a feature used to train the model, but is still used to identify the transactions. For this reason, we will send it as metadata with the request, and it will be stored in the OpenScale payload log.

In [ ]:
scoring_payload = {
    "fields": data['fields'][1:],
    "values": [],
    "meta":{
        "fields": ["ID"],
        "values": []
    }
}

Randomly select fields from the data to score, and randomly assign an integer ID to each.

In [ ]:
import random
import string

letters = string.digits

for _ in range(0, random.randint(MIN_PAYLOADS, MAX_PAYLOADS)):
    value_to_score = random.choice(data['values'])
    scoring_payload['values'].append(value_to_score[1:])
    scoring_payload['meta']['values'].append([int(''.join(random.choices(letters, k=8)))])

Send the scoring request to the model.

In [ ]:
predictions = client.deployments.score(scoring_url, scoring_payload)

Print out the returned predictions.

In [ ]:
predictions

# Congratulations!

If all cells have run successfully, you have submitted data to your model. You can save a version of this notebook using the **File** menu above, and schedule it to run periodically to continue sending data for OpenScale to monitor.